# 01 – EDA: Trip Duration Prediction

**هدف النوتبوك**: أفهم البيانات كويس قبل أي نمذجة.

## أسئلة مبدئية:
- شكل توزيع `trip_duration` عامل إزاي؟ هل في قيم شاذة جدًا؟
- هل في قيم ناقصة/أنواع أعمدة غير متوقعة؟
- نطاق الإحداثيات (lat/lon) معقول؟
- العلاقة المبدئية بين المسافة ومدة الرحلة؟
- تأثير الوقت (ساعة/يوم/شهر)؟

## قائمة المهام (TODO):
- [ ] تحميل عينة من البيانات من `data/raw/`
- [ ] فحوصات سلامة سريعة
- [ ] رسوم/جداول توضيحية قصيرة
- [ ] استنتاجات بنقاط بسيطة أسفل كل جزء


In [ ]:
# ملاحظة: اكتب كود التحميل والتحقق هنا لاحقًا.
# التزم بخطوات الأسئلة فوق – خطوة خطوة.
